In [1]:
import pandas as pd

In [2]:
#Read socioeconomic status per zipcode

In [3]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [4]:
#Read causal effect file(Texas)

In [5]:
tx_treatment_effect = pd.read_csv('tx_climate_causal_effect.csv')

In [6]:
tx_treatment_effect.head()

,zip,1,2,3,4,5,6,elderlyReturns,returnsTotalwSalariesWages,returnsDependentCareCredit,returnsEducationCredit,risk_group,propensity_score,price_shift,y_0,y_0_corrected,y_1,y_1_corrected,treatment_effect,treatment_effect_no_corr
0,71854,0.349521,0.279070,0.134747,0.086183,0.120383,0.030096,0.252394,0.825581,0.273598,0.045828,False,0.316765,4.541389,12.968581,0.634325,12.645249,12.645249,12.010925,-0.323332
1,75001,0.220000,0.231000,0.193000,0.110000,0.166000,0.080000,0.155000,0.864000,0.119000,0.057000,False,0.277153,6.352608,11.136753,4.518273,2.654614,2.654614,-1.863659,-8.482139
2,75002,0.242219,0.162712,0.119877,0.091217,0.226194,0.157781,0.227735,0.844068,0.307242,0.064099,False,0.341157,5.687621,4.104407,6.507428,2.096740,2.096740,-4.410688,-2.007667
3,75006,0.265625,0.281672,0.180321,0.096706,0.133024,0.042652,0.226774,0.826014,0.234797,0.047720,False,0.274645,17.307029,17.350410,17.290603,7.663044,7.663044,-9.627559,-9.687365
4,75007,0.238005,0.214205,0.156403,0.103513,0.207405,0.080468,0.255006,0.834152,0.242539,0.055912,False,0.292541,12.672375,9.308389,14.063412,4.651142,4.651142,-9.412271,-4.657247


In [7]:
#Merge by zip

In [8]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
tx_treatment_effect['zip'] = tx_treatment_effect['zip'].astype(str).str.zfill(5)

In [9]:
tx_socioeconomic_treatment_effect = pd.merge(socioeconomic_df, tx_treatment_effect, on="zip")

In [10]:
#OLS regression treatment effect ~ socioeconomic variables

In [11]:
import statsmodels.api as sm

X = tx_socioeconomic_treatment_effect[['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']]
X = sm.add_constant(X)
y = tx_socioeconomic_treatment_effect['treatment_effect_no_corr']

model = sm.OLS(y, X).fit()
print(model.summary())

                               OLS Regression Results                               
Dep. Variable:     treatment_effect_no_corr   R-squared:                       0.523
Model:                                  OLS   Adj. R-squared:                  0.521
Method:                       Least Squares   F-statistic:                     220.9
Date:                      Thu, 08 May 2025   Prob (F-statistic):          6.83e-128
Time:                              20:45:09   Log-Likelihood:                -2259.7
No. Observations:                       810   AIC:                             4529.
Df Residuals:                           805   BIC:                             4553.
Df Model:                                 4                                         
Covariance Type:                  nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [12]:
# Mean treatment effect

In [13]:
tx_socioeconomic_treatment_effect['treatment_effect_no_corr'].mean()

-2.8664820941090223

In [14]:
# Mean treatment effect: Lower 25% & Upper 25%

In [15]:
low_poverty = tx_socioeconomic_treatment_effect[tx_socioeconomic_treatment_effect['pct_poverty'] <= tx_socioeconomic_treatment_effect['pct_poverty'].quantile(0.25)]

In [16]:
low_poverty['treatment_effect_no_corr'].mean()

-6.390586358983331

In [17]:
high_poverty = tx_socioeconomic_treatment_effect[tx_socioeconomic_treatment_effect['pct_poverty'] >= tx_socioeconomic_treatment_effect['pct_poverty'].quantile(0.75)]

In [18]:
high_poverty['treatment_effect_no_corr'].mean()

2.6553336676162362

In [19]:
low_black = tx_socioeconomic_treatment_effect[tx_socioeconomic_treatment_effect['pct_black'] <= tx_socioeconomic_treatment_effect['pct_black'].quantile(0.25)]

In [20]:
low_black['treatment_effect_no_corr'].mean()

-1.2875402330718055

In [21]:
high_black = tx_socioeconomic_treatment_effect[tx_socioeconomic_treatment_effect['pct_black'] >= tx_socioeconomic_treatment_effect['pct_black'].quantile(0.75)]

In [22]:
high_black['treatment_effect_no_corr'].mean()

-1.2989706301790596

In [23]:
low_college = tx_socioeconomic_treatment_effect[tx_socioeconomic_treatment_effect['pct_college_plus'] <= tx_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.25)]

In [24]:
low_college['treatment_effect_no_corr'].mean()

1.3136812753733718

In [25]:
high_college = tx_socioeconomic_treatment_effect[tx_socioeconomic_treatment_effect['pct_college_plus'] >= tx_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.75)]

In [26]:
high_college['treatment_effect_no_corr'].mean()

-7.173424230510875

In [27]:
low_mortgage = tx_socioeconomic_treatment_effect[tx_socioeconomic_treatment_effect['pct_mortgage'] <= tx_socioeconomic_treatment_effect['pct_mortgage'].quantile(0.25)]

In [28]:
low_mortgage['treatment_effect_no_corr'].mean()

1.7846877594120172

In [29]:
high_mortgage = tx_socioeconomic_treatment_effect[tx_socioeconomic_treatment_effect['pct_mortgage'] >= tx_socioeconomic_treatment_effect['pct_mortgage'].quantile(0.75)]

In [30]:
high_mortgage['treatment_effect_no_corr'].mean()

-6.243841368725559

In [31]:
#Get Texas socioeconomic status

In [32]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [33]:
zipdata = pd.read_excel("uszips.xlsx")

In [34]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
zipdata['zip'] = zipdata['zip'].astype(str).str.zfill(5)

In [35]:
texas_zipdata = zipdata[zipdata['state_name'] == 'Texas']

In [36]:
tx_socioeconomic = pd.merge(socioeconomic_df, texas_zipdata, on="zip")

In [37]:
tx_socioeconomic = tx_socioeconomic.dropna(subset=['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage'])

In [38]:
#Cluster zip using socioeconomic variables

In [39]:
from sklearn.preprocessing import StandardScaler

In [40]:
features = ['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']

In [41]:
scaler = StandardScaler()
tx_socioeconomic_scaled = scaler.fit_transform(tx_socioeconomic[features])

In [42]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [43]:
#Clustering
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in range_n_clusters: #try different numbers of clusters
    kmeans = KMeans(n_clusters=k, random_state=324)
    cluster_labels = kmeans.fit_predict(tx_socioeconomic_scaled)
    #report average Silhouette score
    silhouette_avg = silhouette_score(tx_socioeconomic_scaled, cluster_labels)
    print("For n_clusters ={},".format(k)+" the average silhouette_score is :{}".format(silhouette_avg))

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =2, the average silhouette_score is :0.317295754853063
For n_clusters =3, the average silhouette_score is :0.34690824294218653


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =4, the average silhouette_score is :0.2976261520877028
For n_clusters =5, the average silhouette_score is :0.2447749637520147


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =6, the average silhouette_score is :0.23056240619987858
For n_clusters =7, the average silhouette_score is :0.24809203937343863


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =8, the average silhouette_score is :0.24210801837549212
For n_clusters =9, the average silhouette_score is :0.24731556356001957


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =10, the average silhouette_score is :0.22482243811778432


In [44]:
kmeans = KMeans(n_clusters=3, random_state=324)
cluster_labels = kmeans.fit_predict(tx_socioeconomic_scaled)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [45]:
tx_socioeconomic['cluster'] = cluster_labels

In [46]:
cluster_profile = tx_socioeconomic.groupby('cluster')[features].mean()
print(cluster_profile)

         pct_poverty  pct_black  pct_college_plus  pct_mortgage
cluster                                                        
0           0.163953   0.040533          0.160540      0.357306
1           0.215947   0.377128          0.170100      0.467484
2           0.077369   0.073625          0.436638      0.643711


In [47]:
tx_socioeconomic

,zip,pct_poverty,pct_black,pct_college_plus,pct_mortgage,lat,lng,city,state_id,state_name,...,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone,cluster
0,77831,0.179931,0.036175,0.181982,0.504107,30.73144,-95.90933,Bedias,TX,Texas,...,5.4,48185,Grimes,"{""48185"": 76.53, ""48471"": 22.23, ""48313"": 1.24}",Grimes|Walker|Madison,48185|48471|48313,False,False,America/Chicago,0
1,77837,0.213080,0.457336,0.124043,0.291581,31.00704,-96.67718,Calvert,TX,Texas,...,4.4,48395,Robertson,"{""48395"": 100}",Robertson,48395,False,False,America/Chicago,1
2,77840,0.470817,0.092486,0.516263,0.446392,30.61150,-96.32332,College Station,TX,Texas,...,2138.2,48041,Brazos,"{""48041"": 100}",Brazos,48041,False,False,America/Chicago,0
3,77969,0.000000,0.000000,0.120930,0.589041,28.77579,-96.64719,La Salle,TX,Texas,...,1.7,48239,Jackson,"{""48239"": 100}",Jackson,48239,False,False,America/Chicago,0
4,77990,0.154131,0.018496,0.198400,0.351111,28.48098,-96.93438,Tivoli,TX,Texas,...,2.3,48391,Refugio,"{""48391"": 72.18, ""48057"": 27.82}",Refugio|Calhoun,48391|48057,False,False,America/Chicago,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,76825,0.166120,0.012594,0.166280,0.465451,31.09493,-99.39329,Brady,TX,Texas,...,5.3,48307,McCulloch,"{""48307"": 97.25, ""48319"": 2.75}",McCulloch|Mason,48307|48319,False,False,America/Chicago,0
1921,78219,0.239317,0.311503,0.122646,0.388785,29.44137,-98.38459,San Antonio,TX,Texas,...,416.2,48029,Bexar,"{""48029"": 100}",Bexar,48029,False,False,America/Chicago,1
1922,78223,0.202419,0.043065,0.118245,0.583447,29.31102,-98.38675,San Antonio,TX,Texas,...,540.8,48029,Bexar,"{""48029"": 91.62, ""48493"": 8.38}",Bexar|Wilson,48029|48493,False,False,America/Chicago,0
1923,78225,0.294126,0.001797,0.087727,0.374692,29.38795,-98.52586,San Antonio,TX,Texas,...,2640.6,48029,Bexar,"{""48029"": 100}",Bexar,48029,False,False,America/Chicago,0


In [48]:
tx_socioeconomic.to_csv("tx_socioeconomic.csv", index=False)